In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
checkpoint_path = '/content/drive/MyDrive/train/outputs/BMT_NND_WM110.tar'

# Инициализация

In [ ]:
!pip install fvcore jsonpickle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 6.9 MB/s 
     |████████████████████████████████| 40 kB 6.4 MB/s 
     |████████████████████████████████| 42 kB 1.2 MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221213-py3-none-any.whl size=61498 sha256=00fea8cabff6d19d97cdf6ad049297ef4dfc24aa71b28ab88419c77ec9a443d1
  Stored in directory: /root/.cache/pip/wheels/14/6d/5c/4fd3efe9b62aeae1e7e68204b54487df288e58e28f3d13fa1e
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31547 sha256=3919d4e6d79d070b94c780e3ca7e90404390e011b0539300dfc2448bd83cd29d
  Stored in directory: /root/.cache/pip/wheels/89/3e/24/0f349c0b2eeb6965903035f3b00dbb5c9bea437b4a2f18d82c
Successfully built fvcore iopath


In [ ]:
%cd /content
!git clone -b develop --single-branch --depth 1 'https://github.com/maloxit/BMT.git'
%cd /content/BMT
!git checkout develop

/content
Cloning into 'BMT'...
remote: Enumerating objects: 2103, done.
remote: Counting objects: 100% (2103/2103), done.
remote: Compressing objects: 100% (2091/2091), done.
remote: Total 2103 (delta 12), reused 2064 (delta 9), pack-reused 0
Receiving objects: 100% (2103/2103), 235.62 MiB | 15.66 MiB/s, done.
Resolving deltas: 100% (12/12), done.
Checking out files: 100% (2066/2066), done.
/content/BMT
Already on 'develop'
Your branch is up to date with 'origin/develop'.


In [ ]:
%cd /content/BMT
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.utils.data as torchdt
from itertools import chain
import os
import cv2
import numpy as np

from model.option import get_opts
from model.dataset import MakeupDataset
from model.loss import SAATDLoss, SAATGLoss
from model.model import get_generator, get_dis_non_makeup, get_dis_makeup

from tqdm import tqdm

/content/BMT


In [ ]:
def get_torch_device(opts):
    if opts.platform == 'GPU' and torch.cuda.is_available():
        return torch.device('cuda:{}'.format(opts.device_id))
    else:
        return torch.device('cpu')

In [ ]:
import argparse
opts = argparse.Namespace(**{
    'warp_dir': '/content/BMT/datasets/train/images/warp_tmp',
    'warp_alt_dir': '/content/BMT/datasets/train/images/warp',
    'warp_storage_dir': '/content/drive/MyDrive/train/warp',
    'subset_config_files': 
    [
        'datasets/train_makeup_blue_shades.json',
        'datasets/train_makeup_dark_shades.json',
        'datasets/train_makeup_new_makeup.json',
        'datasets/train_makeup_other.json',
        'datasets/train_makeup_purpule_shades.json',
        'datasets/train_makeup_red_eyes.json',
        'datasets/train_non_makeup.json'
    ],
    'input_dim': 3,
    'output_dim': 3,
    'semantic_dim': 18,
    'batch_size': 1,
    'resize_size': 286,
    'crop_size': 256,
    'flip': True,
    'nThreads': 0,

    # platform related
    'platform': 'GPU',
    'device_id': 0,
    'device_num': 1,

    # ouptput related
    'name': 'BMT_NND_WM',
    'outputs_dir': '/content/drive/MyDrive/train/outputs/',
    'print_iter': 1,
    'save_imgs': True,
    'save_checkpoint_epochs': 5,

    # weight
    'gan_mode': 'lsgan',
    'rec_weight': 1,
    'CP_weight': 0.05,
    'GP_weight': 0.025,
    'cycle_weight': 1,
    'adv_weight': 1,
    'latent_weight': 0.1,
    'semantic_weight': 1,

    # training related
    'init_type': 'normal',
    'init_gain': 0.02,
    'beta1': 0.5,
    'beta2': 0.999,

    'dis_scale': 3,
    'dis_norm': 'None',
    'dis_spectral_norm': True,
    'lr_policy': 'lambda',
    'max_epoch': 1000,
    'n_epochs': 1000,
    'n_epochs_decay': 500,

    'resume': None,
    'num_residule_block': 4,
    'lr': 0.0002
})
test_opts = argparse.Namespace(**{
    'warp_dir': '/content/BMT/datasets/test/images/warp_tmp',
    'warp_alt_dir': '/content/BMT/datasets/test/images/warp',
    'warp_storage_dir': '/content/drive/MyDrive/test/warp',
    'subset_config_files': 
    [
        'datasets/test_makeup.json',
        'datasets/test_non_makeup.json'
    ],
    'input_dim': 3,
    'output_dim': 3,
    'semantic_dim': 18,
    'batch_size': 1,
    'resize_size': 286,
    'crop_size': 256,
    'flip': True,
    'nThreads': 0,

    # platform related
    'platform': 'GPU',
    'device_id': 0,
    'device_num': 1,

    # ouptput related
    'name': 'BMT_ND_WM',
    'outputs_dir': '/content/drive/MyDrive/test/outputs/',
    'print_iter': 1,
    'save_imgs': True,
    'save_checkpoint_epochs': 5,

    # weight
    'gan_mode': 'lsgan',
    'rec_weight': 1,
    'CP_weight': 0.05,
    'GP_weight': 0.025,
    'cycle_weight': 1,
    'adv_weight': 1,
    'latent_weight': 0.1,
    'semantic_weight': 1,

    # training related
    'init_type': 'normal',
    'init_gain': 0.02,
    'beta1': 0.5,
    'beta2': 0.999,

    'dis_scale': 3,
    'dis_norm': 'None',
    'dis_spectral_norm': True,
    'lr_policy': 'lambda',
    'max_epoch': 1000,
    'n_epochs': 1000,
    'n_epochs_decay': 500,

    'resume': None,
    'num_residule_block': 4,
    'lr': 0.0002
})

device = get_torch_device(opts)

In [ ]:
# model G and D
G = get_generator(opts, device)
D_non_makeup = get_dis_non_makeup(opts, device)
D_makeup = get_dis_makeup(opts, device)

# loss G and D
loss_G = SAATGLoss(opts, G, D_non_makeup, D_makeup)
loss_D = SAATDLoss(opts, D_non_makeup, D_makeup)

# optimizer G and D
optimizer_G = torch.optim.Adam(G.parameters(), opts.lr, betas=(opts.beta1, opts.beta2), weight_decay=0.0001)
optimizer_D = torch.optim.Adam(chain(D_non_makeup.parameters(), D_makeup.parameters()), opts.lr,
                                betas=(opts.beta1, opts.beta2), weight_decay=0.0001)

last_epoch = -1

initialize network with normal
initialize network with normal
initialize network with normal


In [ ]:
# Load checkpoint

def load_checkpoint(checkpoint_path, G=None, D_non_makeup=None, D_makeup=None, optimizer_G=None, optimizer_D=None):
    checkpoint = torch.load(checkpoint_path, map_location=device)
    if G is not None:
        G.load_state_dict(checkpoint['model_G_state_dict'])
    if D_non_makeup is not None:
        D_non_makeup.load_state_dict(checkpoint['model_D_non_makeup_state_dict'])
    if D_makeup is not None:
        D_makeup.load_state_dict(checkpoint['model_D_makeup_state_dict'])
    if optimizer_G is not None:
        optimizer_G.load_state_dict(checkpoint['optimizer_G_state_dict'])
    if optimizer_D is not None:
        optimizer_D.load_state_dict(checkpoint['optimizer_D_state_dict'])
    last_epoch = checkpoint['epoch']
    last_g_loss = checkpoint['g_loss']
    last_d_loss = checkpoint['d_loss']

    print("Loading checkpoint")
    print("epoch:\t", last_epoch)
    print("g_loss\t", last_g_loss)
    print("d_loss\t", last_d_loss)
    print("\n")
    return last_epoch

# Загрузка сгенерированной части датасета

In [ ]:
%cd /content/drive/MyDrive/train
!tar -xPf new_warp1.tar -C /content/BMT/datasets/train/images
!tar -xPf new_warp2.tar -C /content/BMT/datasets/train/images
!tar -xPf new_warp3.tar -C /content/BMT/datasets/train/images
!tar -xPf new_warp4.tar -C /content/BMT/datasets/train/images
!tar -xPf new_warp5.tar -C /content/BMT/datasets/train/images
!tar -xPf new_warp6.tar -C /content/BMT/datasets/train/images
!tar -xPf new_warp7.tar -C /content/BMT/datasets/train/images
!tar -xPf new_warp8.tar -C /content/BMT/datasets/train/images
!tar -xPf new_warp9.tar -C /content/BMT/datasets/train/images
!tar -xPf new_warp10.tar -C /content/BMT/datasets/train/images
!tar -xPf new_warp11.tar -C /content/BMT/datasets/train/images
!tar -xPf new_warp12.tar -C /content/BMT/datasets/train/images
!tar -xPf new_warp13.tar -C /content/BMT/datasets/train/images
!tar -xPf new_warp14.tar -C /content/BMT/datasets/train/images
!tar -xPf new_warp15.tar -C /content/BMT/datasets/train/images
!tar -xPf new_warp16.tar -C /content/BMT/datasets/train/images
!tar -xPf new_warp17.tar -C /content/BMT/datasets/train/images
!tar -xPf new_warp18.tar -C /content/BMT/datasets/train/images
!tar -xPf new_warp19.tar -C /content/BMT/datasets/train/images
!tar -xPf new_warp20.tar -C /content/BMT/datasets/train/images
!tar -xPf new_warp21.tar -C /content/BMT/datasets/train/images
!tar -xPf new_warp22.tar -C /content/BMT/datasets/train/images
!tar -xPf new_warp23.tar -C /content/BMT/datasets/train/images
!tar -xPf new_warp24.tar -C /content/BMT/datasets/train/images
!tar -xPf new_warp25.tar -C /content/BMT/datasets/train/images
!tar -xPf new_warp26.tar -C /content/BMT/datasets/train/images
!tar -xPf new_warp27.tar -C /content/BMT/datasets/train/images
!tar -xPf new_warp28.tar -C /content/BMT/datasets/train/images
!tar -xPf new_warp29.tar -C /content/BMT/datasets/train/images

/content/drive/MyDrive/train


# Тест

In [ ]:
def write_test_pair_img(result_dir, test_pair_img, iter):
    if not os.path.exists(result_dir):
        os.makedirs(result_dir)
    img_filename = '%s/gen_%05d.png' % (result_dir, iter)
    torchvision.utils.save_image(test_pair_img / 2 + 0.5, img_filename, nrow=1)


def normalize_image(x):
    return x[:, 0:3, :, :]


def get_img(non_makeup, makeup, z_transfer, z_removal, transfer_g, removal_g, z_rec_non_makeup, z_rec_makeup, z_cycle_non_makeup, z_cycle_makeup):
    #non_makeup_down = normalize_image(F.interpolate(non_makeup, scale_factor=0.25, mode='nearest'))
    #n, c, h, w = non_makeup_down.shape
    #non_makeup_down_warp = torch.bmm(non_makeup_down.view(n, c, h * w), mapY)  # n*HW*1
    #non_makeup_down_warp = non_makeup_down_warp.view(n, c, h, w)
    #non_makeup_warp = F.interpolate(non_makeup_down_warp, scale_factor=4)

    #makeup_down = normalize_image(F.interpolate(makeup, scale_factor=0.25, mode='nearest'))
    #n, c, h, w = makeup_down.shape
    #makeup_down_warp = torch.bmm(makeup_down.view(n, c, h * w), mapX)  # n*HW*1
    #makeup_down_warp = makeup_down_warp.view(n, c, h, w)
    #makeup_warp = F.interpolate(makeup_down_warp, scale_factor=4)

    n, c, h, w = non_makeup.shape

    canvas = torch.ones([1, c, int(h * 4.5), int(w * 4)])


    images_non_makeup = normalize_image(non_makeup).detach()
    images_makeup = normalize_image(makeup).detach()
    images_z_transfer = normalize_image(z_transfer).detach()
    images_z_removal = normalize_image(z_removal).detach()
    images_g_transfer = normalize_image(transfer_g).detach()
    images_g_removal = normalize_image(removal_g).detach()
    images_z_rec_non_makeup = normalize_image(z_rec_non_makeup).detach()
    images_z_rec_makeup = normalize_image(z_rec_makeup).detach()
    images_z_cycle_non_makeup = normalize_image(z_cycle_non_makeup).detach()
    images_z_cycle_makeup = normalize_image(z_cycle_makeup).detach()

    canvas[0:1, :, int(h * 1): int(h * 1) + h, int(w * 0):int(w * 0)+w] = images_non_makeup[0:1, ::]
    
    canvas[0:1, :, int(h * 2.5): int(h * 2.5) + h, int(w * 0):int(w * 0)+w] = images_makeup[0:1, ::]
    
    canvas[0:1, :, int(h * 1): int(h * 1) + h, int(w * 1):int(w * 1)+w] = images_z_transfer[0:1, ::]
    
    canvas[0:1, :, int(h * 2.5): int(h * 2.5) + h, int(w * 1):int(w * 1)+w] = images_z_removal[0:1, ::]

    canvas[0:1, :, int(h * 1): int(h * 1) + h, int(w * 2):int(w * 2)+w] = images_z_cycle_non_makeup[0:1, ::]
    
    canvas[0:1, :, int(h * 2.5): int(h * 2.5) + h, int(w * 2):int(w * 2)+w] = images_z_cycle_makeup[0:1, ::]

    canvas[0:1, :, int(h * 1): int(h * 1) + h, int(w * 3):int(w * 3)+w] = images_z_rec_non_makeup[0:1, ::]
    
    canvas[0:1, :, int(h * 2.5): int(h * 2.5) + h, int(w * 3):int(w * 3)+w] = images_z_rec_makeup[0:1, ::]
    
    canvas[0:1, :, int(h * 0): int(h * 0) + h, int(w * 1):int(w * 1)+w] = images_g_transfer[0:1, ::]
    
    canvas[0:1, :, int(h * 3.5): int(h * 3.5) + h, int(w * 1):int(w * 1)+w] = images_g_removal[0:1, ::]



    #row1 = torch.cat((images_non_makeup[0:1, ::],images_makeup[0:1, ::], makeup_warp[0:1, ::], images_z_transfer[0:1, ::], images_z_removal[0:1, ::], images_g_transfer[0:1, ::], images_g_removal[0:1, ::]), 3)
    return canvas


%cd /content/BMT
checkpoint_path2 = '/content/drive/MyDrive/train/outputs/BMT_ND_WM240.tar'

#dataset = MakeupDataset(opts, device, ['datasets/train_non_makeup.json', 'datasets/train_makeup_new_makeup.json'], transform=False, need_pgt=True, all_comb=False)
dataset = MakeupDataset(test_opts, device, ['datasets/test_non_makeup.json', 'datasets/test_makeup.json'], transform=False, need_pgt=True, all_comb=True)
data_loader = torchdt.DataLoader(dataset, batch_size=1, shuffle=False, num_workers=opts.nThreads)

last_epoch = load_checkpoint(checkpoint_path, G)
#last_epoch = load_checkpoint(checkpoint_path2, G)
G.eval()

with torch.no_grad():
    for i, data in enumerate(tqdm(data_loader)):
        non_makeup = data['non_makeup'].to(device)
        makeup = data['makeup'].to(device)
        transfer_g = data['transfer'].to(device)
        removal_g = data['removal'].to(device)
        non_makeup_parse = data['non_makeup_parse'].to(device)
        makeup_parse = data['makeup_parse'].to(device)

        z_transfer, z_removal, z_rec_non_makeup, z_rec_makeup, z_cycle_non_makeup, z_cycle_makeup, _, _ =\
            G(non_makeup, makeup, non_makeup_parse, makeup_parse)
        #img = get_img(non_makeup, makeup, mapX, mapY, z_transfer, z_removal, transfer_g, removal_g)
        img = get_img(non_makeup, makeup, z_transfer, z_removal, transfer_g, removal_g, z_rec_non_makeup, z_rec_makeup, z_cycle_non_makeup, z_cycle_makeup)
        write_test_pair_img(test_opts.outputs_dir + 'res{}'.format(last_epoch), img, i)

/content/BMT
non_makeup size: 2 makeup size: 12
Loading checkpoint
epoch:	 110
g_loss	 -2.249089557329813
d_loss	 0.4295520579814914




100%|██████████| 24/24 [01:24<00:00,  3.51s/it]


In [ ]:
%cd /content/BMT

checkpoint_path1 = '/content/drive/MyDrive/train/outputs/BMT_NND_WM40.tar'
checkpoint_path2 = '/content/drive/MyDrive/train/outputs/BMT_ND_WM240.tar'
#checkpoint_path2 = '/content/drive/MyDrive/train/outputs/BMT_NND_WM85.tar'

dataset = MakeupDataset(test_opts, device, test_opts.subset_config_files, transform=False, need_pgt=True, all_comb=True)
data_loader = torchdt.DataLoader(dataset, batch_size=1, shuffle=False, num_workers=opts.nThreads)

G_old = get_generator(opts, device)
last_epoch1 = load_checkpoint(checkpoint_path1, G)
last_epoch2 = load_checkpoint(checkpoint_path2, G_old)
G.eval()
G_old.eval()
refs = []
trans1 = []
trans2 = []
pgts = []
bases = []

with torch.no_grad():
    for i, data in enumerate(tqdm(data_loader)):
        non_makeup = data['non_makeup'].to(device)
        makeup = data['makeup'].to(device)
        transfer_g = data['transfer'].to(device)
        removal_g = data['removal'].to(device)
        non_makeup_parse = data['non_makeup_parse'].to(device)
        makeup_parse = data['makeup_parse'].to(device)

        base_index = i // dataset.makeup_size
        ref_index = i % dataset.makeup_size
        if ref_index == 0:
            bases.append(non_makeup[0].detach().cpu() / 2 + 0.5)
            refs.append([])
            trans1.append([])
            trans2.append([])
            pgts.append([])
        refs[base_index].append(makeup[0].detach().cpu() / 2 + 0.5)
        pgts[base_index].append(transfer_g[0].detach().cpu() / 2 + 0.5)
        z_transfer1, z_removal1 =\
            G.get_transfers(non_makeup, makeup, non_makeup_parse, makeup_parse)
        z_transfer2, z_removal2 =\
            G_old.get_transfers(non_makeup, makeup, non_makeup_parse, makeup_parse)
        trans1[base_index].append(z_transfer1[0].detach().cpu() / 2 + 0.5)
        trans2[base_index].append(z_transfer2[0].detach().cpu() / 2 + 0.5)
    nrow = 6
    plain = torch.ones_like(bases[0])
    for i in range(dataset.non_makeup_size):
        for j in range((dataset.makeup_size + nrow - 1) // nrow):
            imgs = []
            imgs.append(bases[i])
            imgs += refs[i][j*nrow:min((j+1)*nrow, dataset.makeup_size)]
            imgs.append(plain)
            imgs += trans2[i][j*nrow:min((j+1)*nrow, dataset.makeup_size)]
            imgs.append(plain)
            imgs += trans2[i][j*nrow:min((j+1)*nrow, dataset.makeup_size)]
            imgs.append(plain)
            imgs += trans1[i][j*nrow:min((j+1)*nrow, dataset.makeup_size)]
            imgs.append(plain)
            imgs += pgts[i][j*nrow:min((j+1)*nrow, dataset.makeup_size)]
            result_dir = test_opts.outputs_dir + 'res{}_{}'.format(last_epoch1, last_epoch2)
            if not os.path.exists(result_dir):
                os.makedirs(result_dir)
            img_filename = '%s/table_%05d.png' % (result_dir, i * 10 + j)
            torchvision.utils.save_image(imgs, img_filename, nrow=1 + min(nrow, dataset.makeup_size - j*nrow))

/content/BMT
non_makeup size: 2 makeup size: 12
initialize network with normal
Loading checkpoint
epoch:	 40
g_loss	 -2.0819977156321214
d_loss	 0.4374274258812266


Loading checkpoint
epoch:	 240
g_loss	 -2.3929487947622934
d_loss	 0.43149305939674387




100%|██████████| 24/24 [00:09<00:00,  2.59it/s]


# Обучение

In [ ]:
%cd /content/BMT

dataset = MakeupDataset(opts, device, opts.subset_config_files, transform=True, need_pgt=True, all_comb=False)
data_loader = torchdt.DataLoader(dataset, batch_size=1, shuffle=True, num_workers=opts.nThreads)

accumulation_steps = 1

last_epoch = load_checkpoint(checkpoint_path, G, D_non_makeup, D_makeup, optimizer_G, optimizer_D)
#last_epoch = load_checkpoint(checkpoint_path, G, D_non_makeup, D_makeup)
G.train()
D_non_makeup.train()
D_makeup.train()
optimizer_G.zero_grad()
optimizer_D.zero_grad()


class LossStatState:
    def __init__(self):
        self.d_loss_mean = 0.0
        self.g_loss_mean = 0.0
        self.loss_G_GAN_mean = 0.0
        self.loss_G_rec_mean = 0.0
        self.loss_G_cycle_mean = 0.0
        self.loss_G_semantic_mean = 0.0
        self.loss_G_SPL_mean = 0.0

    def print_diff(self, prev_state):
        print("g_loss    :\t", self.g_loss_mean, "\tdiff:\t", self.g_loss_mean - prev_state.g_loss_mean)
        print("d_loss    :\t", self.d_loss_mean, "\tdiff:\t", self.d_loss_mean - prev_state.d_loss_mean)
        print("G_GAN     :\t", self.loss_G_GAN_mean, "\tdiff:\t", self.loss_G_GAN_mean - prev_state.loss_G_GAN_mean)
        print("G_rec     :\t", self.loss_G_rec_mean, "\tdiff:\t", self.loss_G_rec_mean - prev_state.loss_G_rec_mean)
        print("G_cycle   :\t", self.loss_G_cycle_mean, "\tdiff:\t", self.loss_G_cycle_mean - prev_state.loss_G_cycle_mean)
        print("G_semantic:\t", self.loss_G_semantic_mean, "\tdiff:\t", self.loss_G_semantic_mean - prev_state.loss_G_semantic_mean)
        print("G_SPL     :\t", self.loss_G_SPL_mean, "\tdiff:\t", self.loss_G_SPL_mean - prev_state.loss_G_SPL_mean)


loss_stat = LossStatState()
for epoch in range(last_epoch + 1, opts.max_epoch):
    loss_stat_prev = loss_stat
    loss_stat = LossStatState()
    
    for i, data in enumerate(tqdm(data_loader)):
        non_makeup = data['non_makeup'].to(device)
        makeup = data['makeup'].to(device)
        transfer_g = data['transfer'].to(device)
        removal_g = data['removal'].to(device)
        non_makeup_parse = data['non_makeup_parse'].to(device)
        makeup_parse = data['makeup_parse'].to(device)

        g_output = G(non_makeup, makeup, non_makeup_parse, makeup_parse)

        z_transfer = g_output[0]
        z_removal = g_output[1]

        d_loss = loss_D(non_makeup, makeup, z_transfer.detach(), z_removal.detach())
        d_loss = d_loss / accumulation_steps
        d_loss.backward()

        loss_D.requires_grad_(False)
        g_loss, loss_distr, _, _, _, _, _, _, _, _ =\
            loss_G(non_makeup, makeup, transfer_g, removal_g, non_makeup_parse, makeup_parse, g_output, weighted=True)
        g_loss = g_loss / accumulation_steps
        g_loss.backward()
        loss_D.requires_grad_(True)

        if (i + 1) % accumulation_steps == 0:
            optimizer_G.step()
            optimizer_D.step()
            optimizer_G.zero_grad()
            optimizer_D.zero_grad()

        loss_stat.d_loss_mean += d_loss.item() * accumulation_steps / len(data_loader)
        loss_stat.g_loss_mean += g_loss.item() * accumulation_steps / len(data_loader)
        loss_stat.loss_G_GAN_mean += loss_distr[0] * g_loss.item() / len(data_loader)
        loss_stat.loss_G_rec_mean += loss_distr[1] * g_loss.item() / len(data_loader)
        loss_stat.loss_G_cycle_mean += loss_distr[2] * g_loss.item() / len(data_loader)
        loss_stat.loss_G_semantic_mean += loss_distr[3] * g_loss.item() / len(data_loader)
        loss_stat.loss_G_SPL_mean += loss_distr[4] * g_loss.item() / len(data_loader)

    if epoch % opts.print_iter == 0:
        print("epoch     :\t", epoch)
        loss_stat.print_diff(loss_stat_prev)
        print("\n")
    if epoch % opts.save_checkpoint_epochs == 0:
        torch.save({
            'epoch': epoch,
            'model_G_state_dict': G.state_dict(),
            'model_D_non_makeup_state_dict': D_non_makeup.state_dict(),
            'model_D_makeup_state_dict': D_makeup.state_dict(),
            'optimizer_G_state_dict': optimizer_G.state_dict(),
            'optimizer_D_state_dict': optimizer_D.state_dict(),
            'g_loss': loss_stat.g_loss_mean,
            'd_loss': loss_stat.d_loss_mean,
            }, opts.outputs_dir + opts.name + str(epoch) + '.tar')
    data_loader.dataset.move_warp_to_storage()

/content/BMT
non_makeup size: 300 makeup size: 183
Loading checkpoint
epoch:	 90
g_loss	 -1.9061052970091505
d_loss	 0.4246068192521732




100%|██████████| 300/300 [09:26<00:00,  1.89s/it]


epoch     :	 91
g_loss    :	 -2.1571429006258653 	diff:	 -2.1571429006258653
d_loss    :	 0.4305417282382648 	diff:	 0.4305417282382648
G_GAN     :	 0.3479786315903652 	diff:	 0.3479786315903652
G_rec     :	 0.06303675756703042 	diff:	 0.06303675756703042
G_cycle   :	 0.11193204536935783 	diff:	 0.11193204536935783
G_semantic:	 0.054135576064745304 	diff:	 0.054135576064745304
G_SPL     :	 -2.734225913408744 	diff:	 -2.734225913408744




100%|██████████| 300/300 [09:09<00:00,  1.83s/it]


epoch     :	 92
g_loss    :	 -2.2037340529759724 	diff:	 -0.046591152350107023
d_loss    :	 0.423829158246517 	diff:	 -0.0067125699917477966
G_GAN     :	 0.3524521783667941 	diff:	 0.004473546776428916
G_rec     :	 0.06087837721919865 	diff:	 -0.0021583803478317717
G_cycle   :	 0.10710371999720725 	diff:	 -0.0048283253721505826
G_semantic:	 0.05210870844696969 	diff:	 -0.002026867617775617
G_SPL     :	 -2.776277040193135 	diff:	 -0.042051126784390824




...

100%|██████████| 300/300 [08:49<00:00,  1.76s/it]


epoch     :	 111
g_loss    :	 -2.2368463798363982 	diff:	 0.012243177493414859
d_loss    :	 0.4301769548654557 	diff:	 0.0006248968839643321
G_GAN     :	 0.3472302705180193 	diff:	 -0.0009083233842616667
G_rec     :	 0.05637694126914703 	diff:	 -0.001000243736884003
G_cycle   :	 0.10311407780835459 	diff:	 -0.00017145727028809732
G_semantic:	 0.048040372053659845 	diff:	 -0.0013194609215785788
G_SPL     :	 -2.7916080438623956 	diff:	 0.015642664966085462




100%|██████████| 300/300 [08:41<00:00,  1.74s/it]


epoch     :	 112
g_loss    :	 -2.245224546194076 	diff:	 -0.008378166357677852
d_loss    :	 0.4332207974791529 	diff:	 0.0030438426136971675
G_GAN     :	 0.3461202899849108 	diff:	 -0.0011099805331085122
G_rec     :	 0.05746541940568265 	diff:	 0.00108847813653562
G_cycle   :	 0.10347357738633904 	diff:	 0.0003594995779844512
G_semantic:	 0.04829652793826003 	diff:	 0.00025615588460018573
G_SPL     :	 -2.8005803586360813 	diff:	 -0.008972314773685763




 92%|█████████▏| 275/300 [08:00<00:43,  1.73s/it]

# Utils

In [ ]:
%cd /content/drive/MyDrive/train
!tar -c -f new_warp30.tar warp

/content/drive/MyDrive/train


In [ ]:
%cp -r /content/drive/MyDrive/train/warp /content/BMT/datasets/train/images

In [ ]:
# Run for dataset generation without training
!python3 /content/BMT/run_generator.py --warp-path '/content/BMT/datasets/test/images/warp_tmp' \
--warp-alt-path '/content/BMT/datasets/test/images/warp' \
--warp-storage '/content/drive/MyDrive/test/warp' \
--storage-every 300 \
--non-makeup-dir '/content/BMT/datasets/test/images/non-makeup' \
--non-makeup-mask-dir '/content/BMT/datasets/test/seg1/non-makeup' \
--non-makeup-lms-dir '/content/BMT/datasets/test/lms/non-makeup' \
--makeup-dir '/content/BMT/datasets/test/images/makeup' \
--makeup-mask-dir '/content/BMT/datasets/test/seg1/makeup' \
--makeup-lms-dir '/content/BMT/datasets/test/lms/makeup' \
--skip-to-index 36300 \
--metadata-only
#--skip-metadata

100% 2/2 [00:06<00:00,  3.15s/it]
100% 24/24 [00:02<00:00,  8.28it/s]


In [ ]:
!du -sh /content/drive/MyDrive/train/warp

123M	/content/drive/MyDrive/train/warp


In [ ]:
!du -sh /content/BMT/datasets/train/images/warp

124M	/content/BMT/datasets/train/images/warp
